# Bayesian Fama-French Factor Model

### Data Settings

In [1]:
import pandas as pd
import numpy as np
import pickle
data_path = 'simulation_data'

start_date = '2016-03-30'
end_date = '2025-03-20'
with open(f"{data_path}/KS200_MASK.pkl", 'rb') as f:
    mask_df:pd.DataFrame = pickle.load(f).ffill(axis=1)
    mask_df = mask_df.loc[:, start_date:end_date]
with open(f"{data_path}/Return.pkl", 'rb') as f:
    returns_df = pickle.load(f).ffill(axis=1) * 0.01
    returns_df = returns_df.loc[:, start_date:end_date]
with open(f"{data_path}/MarketCap.pkl", 'rb') as f:
    mc_df = pickle.load(f).ffill(axis=1)
    mc_df = mc_df.loc[:, start_date:end_date]
with open(f"{data_path}/ifrs-full_Equity.pkl", 'rb') as f:
    be_df = pickle.load(f).ffill(axis=1)
    be_df = be_df.loc[:, start_date:end_date]
with open(f"{data_path}/KOSPI_Close.pkl", 'rb') as f:
    kospi_close_df = pickle.load(f).ffill(axis=1)
    kospi_return_df = kospi_close_df.pct_change(axis=1)
    mkt_df = kospi_return_df.loc[:, start_date:end_date]
with open(f"{data_path}/rf_bond.pkl", 'rb') as f:
    rf_df = pickle.load(f).ffill(axis=1) * 0.01
    rf_df = rf_df.loc[:, start_date:end_date]
with open(f"{data_path}/corp_aa_bond.pkl", 'rb') as f:
    corp_aa_df = pickle.load(f).ffill(axis=1) * 0.01
    corp_aa_df = corp_aa_df.loc[:, start_date:end_date]
with open(f"{data_path}/corp_bb_bond.pkl", 'rb') as f:
    corp_bb_df = pickle.load(f).ffill(axis=1) * 0.01
    corp_bb_df = corp_bb_df.loc[:, start_date:end_date]
with open(f"{data_path}/gov10_bond.pkl", 'rb') as f:
    gov10_df = pickle.load(f).ffill(axis=1) * 0.01
    gov10_df = gov10_df.loc[:, start_date:end_date]
with open(f"{data_path}/gov3_bond.pkl", 'rb') as f:
    gov3_df = pickle.load(f).ffill(axis=1) * 0.01
    gov3_df = gov3_df.loc[:, start_date:end_date]
corp_df = (corp_aa_df + corp_bb_df) / 2

In [4]:
R     = returns_df
MC    = mc_df
BE    = be_df 
RF = rf_df / 252 # 일단위 reblancing 시행하므로 252로 나누어 줌.
GOV10   = gov10_df / 252
GOV3 = gov3_df / 252
CORP  = corp_df / 252
MKT   = mkt_df
mask  = mask_df.astype(bool)

dates = R.columns
tickers = R.index

In [5]:
R

,2016-03-30,2016-03-31,2016-04-01,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-11,2016-04-12,...,2025-03-07,2025-03-10,2025-03-11,2025-03-12,2025-03-13,2025-03-14,2025-03-17,2025-03-18,2025-03-19,2025-03-20
ticker,,,,,,,,,,,,,,,,,,,,,
000030,-0.0137,0.0085,-0.0042,-0.0149,-0.0043,-0.0022,0.0098,-0.0064,0.0043,0.0054,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
000050,0.0106,0.0315,-0.0178,-0.0104,-0.0105,-0.0503,-0.0028,-0.0028,-0.0168,-0.0057,...,0.0031,0.0062,0.0015,0.0000,-0.0031,0.0092,0.0015,-0.0030,0.0015,0.0015
000060,0.0000,-0.0214,0.0219,-0.0092,0.0123,0.0183,-0.0299,0.0247,-0.0181,0.0031,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
000070,-0.0253,0.0227,0.0000,0.0000,0.0159,0.0406,0.0090,-0.0149,-0.0272,-0.0093,...,-0.0067,0.0084,0.0017,-0.0017,-0.0033,0.0067,-0.0033,0.0000,0.0017,0.0083
000080,-0.0328,-0.0203,-0.0675,0.0148,0.0055,0.0109,-0.0234,0.0239,-0.0090,-0.0327,...,0.0005,-0.0047,-0.0042,0.0000,0.0168,-0.0052,0.0078,-0.0031,0.0036,0.0031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.0013,0.0147,-0.0237,-0.0013,-0.0229,-0.0124,-0.0126,0.0127,0.0042,0.0000
454910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0032,-0.0126,0.0000,0.0000,-0.0016,-0.0096,-0.0081,-0.0016,-0.0163,-0.0033
456040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0067,-0.0017,-0.0283,0.0171,-0.0152,-0.0017,0.0188,0.0000,0.0067,0.0050


### Prior Settings
<img align="middle" src="images\bayesian_linear_reg.png" height="400" />

<img align="middle" src="images\g-prior_settings.png" height="400" />

<img align="middle" src="images\model_selection_settings.png" height="400" />